# `mlx-lm`

More info about using `mlx-lm` models with outlines [here](https://dottxt-ai.github.io/outlines/latest/reference/models/mlxlm/)

Table of Contents:
- [JSON Generation](#json-generation)
- [Text Generation](#text-generation)


## JSON Generation

Example based on https://dottxt-ai.github.io/outlines/latest/cookbook/extraction/

In [1]:
from enum import Enum

import jinja2
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from mlx_lm.sample_utils import make_sampler
from pydantic import BaseModel

from outlines_haystack.generators.mlxlm import MLXLMJSONGenerator

In [2]:
class Pizza(str, Enum):
    margherita = "Margherita"
    calzone = "Calzone"


class Order(BaseModel):
    pizza: Pizza
    number: int

In [3]:
prompt_template = """You are the owner of a pizza parlor. Customers \
send you orders from which you need to extract:

1. The pizza that is ordered
2. The number of pizzas

# EXAMPLE

ORDER: I would like one Margherita pizza
RESULT: {"pizza": "Margherita", "number": 1}

# OUTPUT INSTRUCTIONS

Answer in valid JSON. Here are the different objects relevant for the output:

Order:
    pizza (str): name of the pizza
    number (int): number of pizzas

Return a valid JSON of type "Order"

# OUTPUT

ORDER: {{ order }}
RESULT: """

In [4]:
generator = MLXLMJSONGenerator(
    model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
    schema_object=Order,
)

### Standalone

In [5]:
generator.warm_up()

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [6]:
# we use Jinja2 to render the template
prompt = jinja2.Template(prompt_template).render(order="Is it possible to have 12 margheritas?")
generator.run(prompt=prompt, generation_kwargs={"sampler": make_sampler(temp=0.5)})

mx.metal.device_info is deprecated and will be removed in a future version. Use mx.device_info instead.


{'structured_replies': ['{"pizza": "Margherita", "number": 12}']}

### In a Pipeline

In [7]:
pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=MLXLMJSONGenerator(
        model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
        schema_object=Order,
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: MLXLMJSONGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [8]:
pipeline.run(
    {
        "prompt_builder": {"order": "Is it possible to have 12 margheritas?"},
        "llm": {"generation_kwargs": {"sampler": make_sampler(temp=0.5)}},
    }
)

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

{'llm': {'structured_replies': ['{"pizza": "Margherita", "number": 12}']}}

## Choice Generation

In [9]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.mlxlm import MLXLMChoiceGenerator

In [10]:
generator = MLXLMChoiceGenerator(
    model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
    choices=["Positive", "Negative"],
)

### Standalone

In [11]:
generator.warm_up()

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [12]:
generator.run(
    prompt="Classify the following statement: 'I love pizza'",
    generation_kwargs={"sampler": make_sampler(temp=0.5)},
)

{'choice': 'Negative'}

### In a Pipeline

In [13]:
prompt_template = "Classify the following statement: '{{statement}}'"

pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=MLXLMChoiceGenerator(
        model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
        choices=["Positive", "Negative"],
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: MLXLMChoiceGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [14]:
pipeline.run(
    {
        "prompt_builder": {"statement": "I love Italian food"},
        "llm": {"generation_kwargs": {"sampler": make_sampler(temp=0.5)}},
    }
)

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

{'llm': {'choice': 'Positive'}}

## Text Generation

In [15]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.mlxlm import MLXLMTextGenerator

In [16]:
generator = MLXLMTextGenerator(
    model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
)

### Standalone

In [17]:
generator.warm_up()

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [18]:
generator.run(
    prompt="What is the capital of Italy?",
    generation_kwargs={"sampler": make_sampler(temp=0.5)},
)

{'replies': ['Rome\nRome is the capital of Italy. It is a city located in the central part of the Italian Peninsula, and it is known for its rich history, architecture, and art. The city is home to many famous landmarks, including the Colosseum, the Pantheon, and the Vatican City, which is an independent city-state within Rome.\n\nThe city of Rome has a long and fascinating history, dating back to the 8th century BC. It was the center of the ancient Roman Empire, and many of its ancient ruins and artifacts can still be seen today. Rome is also home to many museums, galleries, and cultural institutions, making it a popular destination for tourists and art lovers.\n\nIn addition to its rich history and cultural attractions, Rome is also known for its delicious food and wine. The city is famous for its pasta dishes, pizza, and other Italian specialties, as well as its extensive wine production. Visitors to Rome can enjoy a wide range of culinary experiences, from traditional trattorias to

### In a Pipeline

In [19]:
prompt_template = "What is the capital of {{country}}?"

pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=MLXLMTextGenerator(
        model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: MLXLMTextGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [20]:
pipeline.run(
    {
        "prompt_builder": {"country": "France"},
        "llm": {"generation_kwargs": {"sampler": make_sampler(temp=0.5), "max_tokens": 20}},
    }
)

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

{'llm': {'replies': ['The answer is Paris, which is the city that has been the center of French culture and politics for']}}